In [ ]:
# Experiment 070: Extrapolation Detection v2
# 
# Key insight: When comparing to ALL solvents, the test solvent is close to ITSELF
# So we need to compare to all solvents EXCEPT the test solvent
# 
# This identifies solvents that are truly "outliers" in the solvent space
# Water, HFIP, Ethylene Glycol might be outliers

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import pdist, cdist
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)
torch.set_default_dtype(torch.double)

print('Imports successful')

In [ ]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
TARGET_LABELS = ["Product 2", "Product 3", "SM"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[TARGET_LABELS]
    return X, Y

def load_features(name="spange_descriptors"):
    features = pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)
    return features

def generate_leave_one_out_splits(X, Y):
    all_solvents = X["SOLVENT NAME"].unique()
    for solvent_name in sorted(all_solvents):
        train_idcs_mask = X["SOLVENT NAME"] != solvent_name
        test_idcs_mask = X["SOLVENT NAME"] == solvent_name
        train_X = X[train_idcs_mask]
        train_Y = Y[train_idcs_mask]
        test_X = X[test_idcs_mask]
        test_Y = Y[test_idcs_mask]
        yield (train_X, train_Y), (test_X, test_Y)

def generate_leave_one_ramp_out_splits(X, Y):
    all_solvents_A = X["SOLVENT A NAME"].unique()
    all_solvents_B = X["SOLVENT B NAME"].unique()
    all_solvents = np.union1d(all_solvents_A, all_solvents_B)
    for solvent_name in sorted(all_solvents):
        train_idcs_mask = (X["SOLVENT A NAME"] != solvent_name) & (X["SOLVENT B NAME"] != solvent_name)
        test_idcs_mask = (X["SOLVENT A NAME"] == solvent_name) | (X["SOLVENT B NAME"] == solvent_name)
        train_X = X[train_idcs_mask]
        train_Y = Y[train_idcs_mask]
        test_X = X[test_idcs_mask]
        test_Y = Y[test_idcs_mask]
        yield (train_X, train_Y), (test_X, test_Y)

# Load Spange descriptors for ALL solvents (global)
SPANGE_DF = load_features('spange_descriptors')
print(f'Spange descriptors shape: {SPANGE_DF.shape}')
print('Data loading functions defined')

In [ ]:
# Analyze which solvents are outliers
# Compute distance from each solvent to all OTHER solvents

solvent_scaler = StandardScaler()
scaled_features = solvent_scaler.fit_transform(SPANGE_DF.values)

# For each solvent, compute mean distance to k nearest OTHER solvents
k = 3
outlier_scores = {}

for i, solvent in enumerate(SPANGE_DF.index):
    # Get all other solvents
    other_indices = [j for j in range(len(SPANGE_DF)) if j != i]
    other_features = scaled_features[other_indices]
    
    # Compute distance to k nearest
    distances = cdist([scaled_features[i]], other_features, metric='euclidean')[0]
    k_nearest_dist = np.sort(distances)[:k].mean()
    outlier_scores[solvent] = k_nearest_dist

# Sort by outlier score
sorted_scores = sorted(outlier_scores.items(), key=lambda x: x[1], reverse=True)
print("Solvent outlier scores (higher = more isolated):")
for solvent, score in sorted_scores:
    print(f"  {solvent}: {score:.4f}")

# Compute mean and std
mean_score = np.mean(list(outlier_scores.values()))
std_score = np.std(list(outlier_scores.values()))
print(f"\nMean: {mean_score:.4f}, Std: {std_score:.4f}")
print(f"Threshold (mean + 1*std): {mean_score + std_score:.4f}")

In [ ]:
from abc import ABC, abstractmethod

class SmilesFeaturizer(ABC):
    def __init__(self):
        raise NotImplementedError
    def featurize(X, Y):
        raise NotImplementedError

class BaseModel(ABC):
    def __init__(self):
        pass
    def train_model(self, X_train, y_train):
        raise NotImplementedError
    def predict(self):
        raise NotImplementedError

In [ ]:
# Featurizers
class PrecomputedFeaturizer(SmilesFeaturizer):
    def __init__(self, features='spange_descriptors'):
        self.features = features
        self.featurizer = load_features(self.features)
        self.feats_dim = self.featurizer.shape[1] + 2

    def featurize(self, X):
        X_numeric = X[INPUT_LABELS_NUMERIC]
        X_smiles_feat = self.featurizer.loc[X["SOLVENT NAME"]]
        X_numeric_tensor = torch.tensor(X_numeric.values)
        X_smiles_feat_tensor = torch.tensor(X_smiles_feat.values)
        X_out = torch.cat((X_numeric_tensor, X_smiles_feat_tensor), dim=1)
        return X_out

class PrecomputedFeaturizerMixed(SmilesFeaturizer):
    def __init__(self, features='spange_descriptors'):
        self.features = features
        self.featurizer = load_features(self.features)
        self.feats_dim = self.featurizer.shape[1] + 2

    def featurize(self, X):
        X_numeric = X[INPUT_LABELS_NUMERIC]
        X_smiles_A_feat = self.featurizer.loc[X["SOLVENT A NAME"]]
        X_smiles_B_feat = self.featurizer.loc[X["SOLVENT B NAME"]]
        X_pct = X["SolventB%"].values.reshape(-1, 1)
        X_smiles_feat = X_smiles_A_feat.values * (1 - X_pct/100) + X_smiles_B_feat.values * (X_pct/100)
        X_numeric_tensor = torch.tensor(X_numeric.values)
        X_smiles_feat_tensor = torch.tensor(X_smiles_feat)
        X_out = torch.cat((X_numeric_tensor, X_smiles_feat_tensor), dim=1)
        return X_out

print('Featurizers defined')

In [ ]:
# Pre-compute outlier scores for all solvents (excluding self)
# This is done ONCE globally

def compute_solvent_outlier_scores(k=3):
    """Compute outlier score for each solvent based on distance to k nearest OTHER solvents."""
    solvent_scaler = StandardScaler()
    scaled_features = solvent_scaler.fit_transform(SPANGE_DF.values)
    
    outlier_scores = {}
    for i, solvent in enumerate(SPANGE_DF.index):
        other_indices = [j for j in range(len(SPANGE_DF)) if j != i]
        other_features = scaled_features[other_indices]
        distances = cdist([scaled_features[i]], other_features, metric='euclidean')[0]
        k_nearest_dist = np.sort(distances)[:k].mean()
        outlier_scores[solvent] = k_nearest_dist
    
    return outlier_scores

SOLVENT_OUTLIER_SCORES = compute_solvent_outlier_scores(k=3)
mean_outlier_score = np.mean(list(SOLVENT_OUTLIER_SCORES.values()))
std_outlier_score = np.std(list(SOLVENT_OUTLIER_SCORES.values()))

print(f"Pre-computed outlier scores. Mean: {mean_outlier_score:.4f}, Std: {std_outlier_score:.4f}")

In [ ]:
# Extrapolation-Aware MLP Model v2
# Uses pre-computed outlier scores to identify solvents that need conservative predictions

class ExtrapolationAwareMLPModelV2(nn.Module, BaseModel):
    def __init__(self, features='spange_descriptors', hidden_dims=[64, 64], output_dim=3, 
                 dropout=0.0, data='single', blend_threshold=1.5):
        super(ExtrapolationAwareMLPModelV2, self).__init__()
        self.data_type = data
        self.blend_threshold = blend_threshold  # Number of std devs above mean to start blending
        
        if data == 'single':
            self.smiles_featurizer = PrecomputedFeaturizer(features=features)
        else:
            self.smiles_featurizer = PrecomputedFeaturizerMixed(features=features)
        
        input_dim = self.smiles_featurizer.feats_dim
        prev_dim = input_dim
        layers = []
        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.ReLU())
            if dropout > 0:
                layers.append(nn.Dropout(dropout))
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, output_dim))
        layers.append(nn.Sigmoid())  # Ensure [0,1] output
        self.model = nn.Sequential(*layers)
        
        self.train_Y = None
        self.input_scaler = StandardScaler()
    
    def get_blend_weight(self, solvent_name):
        """Get blend weight based on pre-computed outlier score."""
        score = SOLVENT_OUTLIER_SCORES.get(solvent_name, mean_outlier_score)
        # Normalize: how many std devs above mean?
        z_score = (score - mean_outlier_score) / std_outlier_score
        # Blend weight: 0 for normal solvents, increases for outliers
        blend_weight = np.clip((z_score - self.blend_threshold) / 2.0, 0, 1)
        return blend_weight
    
    def train_model(self, X_train, y_train):
        self.train_Y = y_train.values if hasattr(y_train, 'values') else y_train
        
        X_featurized = self.smiles_featurizer.featurize(X_train)
        X_scaled = self.input_scaler.fit_transform(X_featurized.numpy())
        X_tensor = torch.tensor(X_scaled)
        Y_tensor = torch.tensor(y_train.values if hasattr(y_train, 'values') else y_train)
        
        dataset = TensorDataset(X_tensor, Y_tensor)
        dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
        
        optimizer = torch.optim.Adam(self.parameters(), lr=0.001)
        criterion = nn.MSELoss()
        
        self.train()
        for epoch in range(200):
            for batch_X, batch_Y in dataloader:
                optimizer.zero_grad()
                pred = self.model(batch_X)
                loss = criterion(pred, batch_Y)
                loss.backward()
                optimizer.step()
    
    def predict(self, X):
        self.eval()
        with torch.no_grad():
            X_featurized = self.smiles_featurizer.featurize(X)
            X_scaled = self.input_scaler.transform(X_featurized.numpy())
            X_tensor = torch.tensor(X_scaled)
            raw_pred = self.model(X_tensor).numpy()
        
        # Get blend weights for each sample
        if self.data_type == 'single':
            solvent_names = X["SOLVENT NAME"].values
            blend_weights = np.array([self.get_blend_weight(s) for s in solvent_names])
        else:
            # For mixtures, use max of both solvents' blend weights
            solvent_a_names = X["SOLVENT A NAME"].values
            solvent_b_names = X["SOLVENT B NAME"].values
            blend_weights_a = np.array([self.get_blend_weight(s) for s in solvent_a_names])
            blend_weights_b = np.array([self.get_blend_weight(s) for s in solvent_b_names])
            blend_weights = np.maximum(blend_weights_a, blend_weights_b)
        
        # Compute population mean from training data
        mean_pred = self.train_Y.mean(axis=0)
        
        # Blend: for outliers, move toward mean
        blended = (1 - blend_weights.reshape(-1, 1)) * raw_pred + blend_weights.reshape(-1, 1) * mean_pred
        
        # Debug: print blend weights for first fold
        unique_solvents = X["SOLVENT NAME"].unique() if self.data_type == 'single' else np.union1d(X["SOLVENT A NAME"].unique(), X["SOLVENT B NAME"].unique())
        if len(unique_solvents) == 1:
            solvent = unique_solvents[0]
            print(f"  Solvent: {solvent}, Outlier score: {SOLVENT_OUTLIER_SCORES.get(solvent, 0):.4f}, Blend weight: {blend_weights[0]:.4f}")
        
        return torch.tensor(blended)

print('ExtrapolationAwareMLPModelV2 defined')

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = ExtrapolationAwareMLPModelV2(blend_threshold=1.0) # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

submission_single_solvent = pd.DataFrame(all_predictions)
print(f"Single solvent predictions shape: {submission_single_solvent.shape}")

# Calculate CV score for single solvent
X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_mse = []
for fold_idx, split in enumerate(split_generator):
    (train_X, train_Y), (test_X, test_Y) = split
    fold_preds = submission_single_solvent[submission_single_solvent['fold'] == fold_idx]
    pred_values = fold_preds[['target_1', 'target_2', 'target_3']].values
    true_values = test_Y.values
    mse = ((pred_values - true_values) ** 2).mean()
    all_mse.append(mse)
print(f"Single solvent CV MSE: {np.mean(all_mse):.6f}")

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = ExtrapolationAwareMLPModelV2(data='full', blend_threshold=1.0) # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

submission_full_data = pd.DataFrame(all_predictions)
print(f"Full data predictions shape: {submission_full_data.shape}")

# Calculate CV score for full data
X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_mse = []
for fold_idx, split in enumerate(split_generator):
    (train_X, train_Y), (test_X, test_Y) = split
    fold_preds = submission_full_data[submission_full_data['fold'] == fold_idx]
    pred_values = fold_preds[['target_1', 'target_2', 'target_3']].values
    true_values = test_Y.values
    mse = ((pred_values - true_values) ** 2).mean()
    all_mse.append(mse)
print(f"Full data CV MSE: {np.mean(all_mse):.6f}")

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("submission.csv", index=True)

# Also save to /home/submission/
import shutil
shutil.copy("submission.csv", "/home/submission/submission.csv")

print(f"Submission saved. Shape: {submission.shape}")
print(f"Predictions range: target_1 [{submission['target_1'].min():.3f}, {submission['target_1'].max():.3f}]")
print(f"Predictions range: target_2 [{submission['target_2'].min():.3f}, {submission['target_2'].max():.3f}]")
print(f"Predictions range: target_3 [{submission['target_3'].min():.3f}, {submission['target_3'].max():.3f}]")

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################